- we don't have a left and right state so can eliminate all the boundary if statement stuff 
    - delete all the left states 
        - then delete all "r" in the right state names 
- U==q?? 
- par_getd gets the values from the input file in the "problem" block
-  Structs in C are like objects/classes but without methods or public/private specifications. "Members" are just instance variables.
- look into athena.h for how the object structs are defined 
- Difference between ConsS and Cons1Ds
    - except Cons1DS doesn't contain $B_x$, not relevant for our case here anyways, but why?/